In [0]:
dbutils.widgets.text('p_file_date', '2021-03-21')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

Find race years for which the data needs to be reprocessed

In [0]:
race_results_list = spark.read.parquet(f"{presentation_folder_path}/race_results") \
    .filter(f"file_date = '{v_file_date}'") \
    .select("race_year") \
    .distinct() \
    .collect()


In [0]:
race_year_list = []
for race_year in race_results_list:
    race_year_list.append(race_year.race_year)


In [0]:
from pyspark.sql.functions import col
race_results = spark.read.parquet(f"{presentation_folder_path}/race_results") \
    .filter(col("race_year").isin(race_year_list))

In [0]:
from pyspark.sql.functions import sum, desc, when, count, col

driver_standings = race_results \
    .groupBy('race_year', 'driver_name', 'driver_nationality', 'team') \
    .agg(sum('points').alias('total_points'), count(when(col('position') == 1, True)).alias('wins')) \
    .orderBy(desc('total_points'))

In [0]:
display(driver_standings)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

driver_rank_spec = Window.partitionBy('race_year').orderBy(desc('total_points'), desc('wins'))
final_df = driver_standings \
    .withColumn('rank', rank().over(driver_rank_spec))
display(final_df)


In [0]:
overwrite_partition(final_df, 'f1_presentation', 'driver_standings', 'race_year')